In [33]:
import os

os.chdir("../")

import json
from time import sleep

import pandas as pd
import requests

from vook_db_lambda.config import MAX_PAGE, REQ_URL, WANT_ITEMS_RAKUTEN, req_params
from vook_db_lambda.utils import (
    DataFrame_maker_rakuten,
    DataFrame_maker_yahoo,
    create_api_input,
    repeat_dataframe_maker,
)

In [34]:
# df_api_input = create_api_input()

In [35]:
cnt = 1
df = pd.DataFrame(columns=WANT_ITEMS_RAKUTEN)

In [41]:
req_params

{'applicationId': '1040242227042771333',
 'affiliateId': '33a2906e.1076e2ef.33a2906f.06927704',
 'format': 'json',
 'formatVersion': '2',
 'keyword': "Levi's 501 BIGE 中古",
 'hits': 30,
 'sort': '-itemPrice',
 'page': 10,
 'minPrice': 100}

In [36]:
req_params["page"] = cnt
brand_name = "Levi's"
line_name = "501"
knowledge_name = "BIGE"
keyword = f"{brand_name} {line_name} {knowledge_name} 中古"  # update1
req_params["keyword"] = keyword

In [43]:
while True:
    req_params["page"] = cnt
    res = requests.get(REQ_URL, req_params)
    res_code = res.status_code
    res = json.loads(res.text)
    print(res)
    if res_code != 200:
        print(
            f"""
        ErrorCode -> {res_code}\n
        Error -> {res['error']}\n
        Page -> {cnt}"""
        )
    else:
        if res["hits"] == 0:
            print("返ってきた商品数の数が0なので、ループ終了")
            break
        tmp_df = pd.DataFrame(res["Items"])[WANT_ITEMS_RAKUTEN]
        df = pd.concat([df, tmp_df], ignore_index=True)
    if cnt == MAX_PAGE:
        print("MAX PAGEに到達したので、ループ終了")
        break
    # logger.info(f"{cnt} end!")
    cnt += 1
    # リクエスト制限回避
    sleep(1)
    print("Finished!!")

{'GenreInformation': [], 'Items': [{'affiliateRate': 4, 'affiliateUrl': 'https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1qd11.g00r3cee.brn1r0ee/?pc=https%3A%2F%2Fitem.rakuten.co.jp%2Fjumblestore%2F2335501481071%2F&m=http%3A%2F%2Fm.rakuten.co.jp%2Fjumblestore%2Fi%2F34063630%2F&rafcid=wsc_i_is_1040242227042771333', 'asurakuArea': '', 'asurakuClosingTime': '', 'asurakuFlag': 0, 'availability': 1, 'catchcopy': '', 'creditCardFlag': 1, 'endTime': '', 'genreId': '551186', 'giftFlag': 0, 'imageFlag': 1, 'itemCaption': '【ブランド】Levi’s Vintage Clothing 【ブランドカナ】リーバイスヴィンテージクロージング 【型番】 【程度】A 【サイズ】29 【メインカラー】インディゴ 【素材・生地】デニム 【キーワード】\u3000\u3000 2024/12/22 セカンドストリート国立店【3550】【中古】Levi’s Vintage Clothing◆タグ付き/BIG E/ボトム ブランド Levi’s Vintage Clothing 型番 カラー インディゴ 柄 素材・生地 デニム&gt;色・素材について サイズ 29&gt;サイズ表示について 実寸 【パンツ】 ウエスト幅：36 / ヒップ幅：42 / 股上：26 / 股下：66 【その他】 その他サイズ：/ 商品は\u3000セカンドストリート国立店の店頭にて販売しております。商品に関するお問合わせは、お電話(TEL:042-580-1635)にて、問合わせ番号：2335501481071をお伝えください。 配送方法やお支払い方法に関するお問い合わせは、サポートセンターまでご連絡をお願いします。 

In [40]:
df.iloc[0].values

array(['【中古】Levi’s◆VT/60s/BIGE/501/ダブルネーム/ストレートパンツ/30/デニム/IDG/90パーセント//【メンズウェア】',
       605660,
       'https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1qd11.g00r3cee.brn1r0ee/?pc=https%3A%2F%2Fitem.rakuten.co.jp%2Fjumblestore%2F2334452250989%2F&m=http%3A%2F%2Fm.rakuten.co.jp%2Fjumblestore%2Fi%2F33173304%2F&rafcid=wsc_i_is_1040242227042771333'],
      dtype=object)

In [ ]:
df["platform_id"] = platform_id
df["knowledge_id"] = knowledge_id
df["size_id"] = size_id
df_main = df.rename(
    columns={"itemName": "name", "itemPrice": "price", "itemUrl": "url"}
)
df_main = df_main.reindex(
    columns=[
        "id",
        "name",
        "url",
        "price",
        "knowledge_id",
        "platform_id",
        "size_id",
    ]
)
print("price type before:", df_main["price"].dtype)
df_main["price"] = df_main["price"].astype(int)
print("price type after:", df_main["price"].dtype)
run_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
df_main["created_at"] = run_time
df_main["updated_at"] = run_time